## Part 1: Preprocessing

In [136]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [137]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [138]:
#Create a copy of the df for backup
attrition_df_backup = attrition_df.copy()

In [139]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]

In [140]:
unique_departments = y_df['Department'].unique()

# Display the unique values
print(unique_departments)

['Sales' 'Research & Development' 'Human Resources']


In [141]:
# Create a list of at least 10 column names to use as X data
column_names = [
    "Education",
    "Age",
    "DistanceFromHome",
    "JobSatisfaction",
    "OverTime",
    "StockOptionLevel",
    "WorkLifeBalance",
    "YearsAtCompany",
    "YearsSinceLastPromotion",
    "NumCompaniesWorked"
]

# Create X_df using your selected columns
X_df = attrition_df[column_names]

# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [142]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [143]:
y_train.head()

,Attrition,Department
1097,No,Research & Development
727,No,Research & Development
254,No,Sales
1175,No,Research & Development
1341,No,Research & Development


In [164]:
department_counts = y_train['Department'].value_counts()

print(department_counts)

Research & Development    765
Sales                     361
Human Resources            50
Name: Department, dtype: int64


In [145]:
# Convert OverTime column to numeric values in the training set
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})

# Convert OverTime column to numeric values in the testing set
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})

# Verify the conversion in the training set
print("Training set OverTime values:", X_train['OverTime'].unique())
print(X_train['OverTime'].head())

# Verify the conversion in the testing set
print("Testing set OverTime values:", X_test['OverTime'].unique())
print(X_test['OverTime'].head())

Training set OverTime values: [0 1]
1097    0
727     0
254     0
1175    0
1341    0
Name: OverTime, dtype: int64
Testing set OverTime values: [0 1]
1041    0
184     0
1222    0
67      0
220     0
Name: OverTime, dtype: int64


In [146]:
print(y_train.columns)
print(y_test.columns)

Index(['Attrition', 'Department'], dtype='object')
Index(['Attrition', 'Department'], dtype='object')


In [147]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_train_scaled = scaler.fit_transform(X_train)

# Scale the training and testing data
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

#Verify the data was transformed correctly
print("Scaled training data:")
print(X_train_scaled.head())
print("\nScaled testing data:")
print(X_test_scaled.head())

Scaled training data:
      Education       Age  DistanceFromHome  JobSatisfaction  OverTime  \
1097  -0.863356 -1.388559          1.440396        -1.582336  -0.63641   
727   -0.863356 -2.040738         -0.522699         1.152834  -0.63641   
254   -0.863356 -0.845077          1.317703         1.152834  -0.63641   
1175   0.099933  0.241886          0.336155        -0.670613  -0.63641   
1341   0.099933 -0.627685          1.317703         0.241111  -0.63641   

      StockOptionLevel  WorkLifeBalance  YearsAtCompany  \
1097          2.547471         0.357435       -0.974263   
727          -0.945525         0.357435       -1.138573   
254           0.218807         0.357435       -0.645643   
1175         -0.945525         0.357435       -0.317023   
1341          0.218807         0.357435        0.504527   

      YearsSinceLastPromotion  NumCompaniesWorked  
1097                -0.676110           -1.059168  
727                 -0.676110           -0.659431  
254                 -0

In [148]:
y_train.head()

,Attrition,Department
1097,No,Research & Development
727,No,Research & Development
254,No,Sales
1175,No,Research & Development
1341,No,Research & Development


In [149]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
department_encoded = encoder.fit_transform(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_encoded_train = encoder.transform(y_train[['Department']])
department_encoded_train_df = pd.DataFrame(department_encoded_train, columns=encoder.get_feature_names_out(['Department']))
y_train_draft = y_train.drop(columns=['Department']).reset_index(drop=True).join(department_encoded_train_df)

department_encoded_test = encoder.transform(y_test[['Department']])
department_encoded_test_df = pd.DataFrame(department_encoded_test, columns=encoder.get_feature_names_out(['Department']))
y_test_draft= y_test.drop(columns=['Department']).reset_index(drop=True).join(department_encoded_test_df)

y_train_draft.head(20)

,Attrition,Department_Research & Development,Department_Sales
0,No,1.0,0.0
1,No,1.0,0.0
2,No,0.0,1.0
3,No,1.0,0.0
4,No,1.0,0.0
5,No,0.0,0.0
6,No,1.0,0.0
7,No,0.0,0.0
8,No,0.0,1.0
9,No,1.0,0.0


In [150]:
y_test_draft.head()

,Attrition,Department_Research & Development,Department_Sales
0,No,0.0,1.0
1,No,1.0,0.0
2,Yes,0.0,0.0
3,No,1.0,0.0
4,No,1.0,0.0


In [151]:
# Create a OneHotEncoder for the Attrition column
encoder = OneHotEncoder(sparse_output=False, drop='first')

# Fit the encoder to the training data
attrition_encoded = encoder.fit_transform(y_train_draft[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data

attrition_encoded_train = encoder.transform(y_train_draft[['Attrition']])
attrition_encoded_train_df = pd.DataFrame(attrition_encoded_train, columns=encoder.get_feature_names_out(['Attrition']))
y_train_final = y_train_draft.drop(columns=['Attrition']).reset_index(drop=True).join(attrition_encoded_train_df)

attrition_encoded_test = encoder.transform(y_test_draft[['Attrition']])
attrition_encoded_test_df = pd.DataFrame(attrition_encoded_test, columns=encoder.get_feature_names_out(['Attrition']))
y_test_final= y_test_draft.drop(columns=['Attrition']).reset_index(drop=True).join(attrition_encoded_test_df)

y_train_final.head()

,Department_Research & Development,Department_Sales,Attrition_Yes
0,1.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [152]:
y_test_final.head()

,Department_Research & Development,Department_Sales,Attrition_Yes
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0


In [153]:
#Engineering some columns for missing values

y_train_final['Department_Human_Resources'] = (
    (y_train_final['Department_Research & Development'] == 0) &
    (y_train_final['Department_Sales'] == 0)
).astype(float)

# Create the Attrition_No column
y_train_final['Attrition_No'] = (
    (y_train_final['Attrition_Yes'] == 0)
).astype(float)

y_test_final['Department_Human_Resources'] = (
    (y_test_final['Department_Research & Development'] == 0) &
    (y_test_final['Department_Sales'] == 0)
).astype(float)

# Create the Attrition_No column
y_test_final['Attrition_No'] = (
    (y_test_final['Attrition_Yes'] == 0)
).astype(float)

y_test_final.head()

,Department_Research & Development,Department_Sales,Attrition_Yes,Department_Human_Resources,Attrition_No
0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0


In [154]:
#Splitting the training set by branches we'll use

y_train_department = y_train_final[['Department_Research & Development', 'Department_Sales', 'Department_Human_Resources']]
y_train_attrition = y_train_final[['Attrition_No', 'Attrition_Yes']]

y_test_department = y_test_final[['Department_Research & Development', 'Department_Sales', 'Department_Human_Resources']]
y_test_attrition = y_test_final[['Attrition_No', 'Attrition_Yes']]

## Create, Compile, and Train the Model

In [155]:
from tensorflow.keras import layers, models, Model

# Find the number of columns in the X training data
num_columns = X_train.shape[1]
# Create the input layer
input_layer = layers.Input(shape=(num_columns,), name='input')

# Create at least two shared layers
shared1 = layers.Dense(64, activation='relu', name='shared1')(input_layer)
shared2 = layers.Dense(128, activation='relu', name='shared2')(shared1)

In [156]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(shared2)

# Create the output layer
department_output = layers.Dense(3, activation='softmax', name='department_output')(department_hidden)


In [157]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(shared2)

# Create the output layer
attrition_output = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden)


In [158]:
# Create the model
model = models.Model(inputs=input_layer, outputs=[department_output, attrition_output])


# Compile the model
model.compile(
    optimizer='adam',
    loss={
        'department_output': 'categorical_crossentropy', 
        'attrition_output': 'binary_crossentropy' 
    },      
    metrics={
        'department_output': ['accuracy'],
        'attrition_output': ['accuracy']
    }
)

# Summarize the model
model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input (InputLayer)  │ (None, 10)        │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared1 (Dense)     │ (None, 64)        │        704 │ input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared2 (Dense)     │ (None, 128)       │      8,320 │ shared1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_hidden   │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_hidden    │ (None, 32)        │      4,128 │ shared2[0][0]     │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │         99 │ department_hidde… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │         66 │ attrition_hidden… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,445 (68.14 KB)

 Trainable params: 17,445 (68.14 KB)

 Non-trainable params: 0 (0.00 B)

In [159]:
# Train the model

model.fit(
    X_train,
    {
        'department_output': y_train_department.values, 
        'attrition_output': y_train_attrition.values
    },
    epochs=100,  # You can adjust the number of epochs based on your needs
    batch_size=32,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)


Epoch 1/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - attrition_output_accuracy: 0.8372 - department_output_accuracy: 0.5386 - loss: 2.3020 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6017 - val_loss: 1.3193
Epoch 2/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8450 - department_output_accuracy: 0.6333 - loss: 1.2576 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6186 - val_loss: 1.3362
Epoch 3/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8582 - department_output_accuracy: 0.6705 - loss: 1.1517 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6102 - val_loss: 1.2842
Epoch 4/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - attrition_output_accuracy: 0.8257 - department_output_accuracy: 0.6269 - loss: 1.2151 - val_attrition_output_accuracy: 0.7966 - val_department_output_accuracy: 0.6314 - val_loss: 1.3520
Epoch 5/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

In [162]:
# Evaluate the model with the testing data
results = model.evaluate(
    np.array(X_test),
    {
        'department_output': y_test_department.values, 
        'attrition_output': y_test_attrition.values
    }
)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - attrition_output_accuracy: 0.8614 - department_output_accuracy: 0.5776 - loss: 1.3951


In [163]:
# Print the accuracy for both department and attrition
print(f"Department accuracy: {results[2]}")
print(f"Attrition accuracy: {results[1]}")

Department accuracy: 0.5918367505073547
Attrition accuracy: 0.8605442047119141


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 

Because the classification data set for 'Department' is imbalanced, as "Human Resources" values are disproportaionaly low. Hence the accuracy of the set may not be the best metric to use on this data. Precision and Recall and F1 scores would likely be the better option as it will give specific insights on how the model performs on each possible classification value.

2. 

Sigmoid Activation for Attrition (Yes/No):

Reason: The sigmoid function outputs a probability between 0 and 1, making it suitable for binary classification. This fits well for a task like predicting whether attrition will occur (Yes/No). The single output neuron with sigmoid activation provides the probability of the positive class.

Softmax Activation for Department (3 Output Possibilities):

Reason: The softmax function is used when you have multiple classes and want the model to output probabilities for each class. It converts raw scores into a probability distribution over the classes. This is suitable for predicting multi class problems.

3. 

Some possiblities to improve the model:

Hyperparameter Tuning: Adjusting hyperparameters like learning rate, batch size, and network architecture improves model performance by finding the optimal settings. Proper tuning balances overfitting and underfitting, leading to better generalization on new data.

Feature Engineering: This involves creating or modifying features to provide more useful information to the model. By adding relevant features or selecting the most important ones, you enhance the model’s ability to make accurate predictions and reduce noise.

Data Augmentation: Generating additional training samples through transformations or synthetic data increases the dataset size and diversity. This helps the model learn more robust features and addresses issues like class imbalance, leading to improved generalization.
